In [9]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array

import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact

In [10]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice])#, cmap=plt.cm.gray)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [11]:
#read rt dose dicom file
rd=dcm.read_file('rd_9e.dcm')
rdpix=rd.pixel_array.astype(np.float32)
rdim=sitk.GetImageFromArray(rdpix)
rdim.SetOrigin([-184.3847655, -119.3847655, -180])
rdim.SetSpacing([2.5, 2.5, 2.5])
sitk.WriteImage(rdim, 'rdim.mhd')

In [12]:
rdimmhd=sitk.ReadImage('rdim.mhd')

In [13]:
gateim=sitk.ReadImage('gate_test.mhd')

In [14]:
print(rdimmhd.GetOrigin())
print(rdimmhd.GetSpacing())
print(rdimmhd.GetSize())
print(gateim.GetOrigin())
print(gateim.GetSpacing())
print(gateim.GetSize())

(-184.3847655, -119.3847655, -180.0)
(2.5, 2.5, 2.5)
(147, 90, 154)
(-134.57, -128.215, -180.0)
(5.0, 5.0, 5.0)
(50, 50, 81)


In [15]:
gatepix=sitk.GetArrayFromImage(gateim)
rdpixmhd=sitk.GetArrayFromImage(rdimmhd)
ct_viewer(gatepix)
ct_viewer(rdpixmhd)

interactive(children=(IntSlider(value=40, description=u'myslice', max=81), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=77, description=u'myslice', max=154), Output()), _dom_classes=('widget-interact',))

In [16]:
rdim_res=sitk.Resample(rdimmhd, gateim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
#rdim_res=sitk.Resample(rdimmhd, [50, 50, 81], sitk.Transform(), sitk.sitkNearestNeighbor, )

In [17]:
rdpixres=sitk.GetArrayFromImage(rdim_res)
ct_viewer(rdpixres)
print(rdim_res.GetOrigin())
print(rdim_res.GetSpacing())
print(rdim_res.GetSize())

interactive(children=(IntSlider(value=40, description=u'myslice', max=81), Output()), _dom_classes=('widget-interact',))

(-134.57, -128.215, -180.0)
(5.0, 5.0, 5.0)
(50, 50, 81)


In [86]:
sitk.WriteImage(rdim_res, 'rdim_res2.mhd')

In [75]:
print(gateim.TransformIndexToPhysicalPoint(((0,0,0))))
print(rdim.TransformIndexToPhysicalPoint(((0,0,0))))
euler3d = sitk.Euler3DTransform()
          # Why do we set the center?
euler3d.SetCenter(gateim.TransformContinuousIndexToPhysicalPoint(np.array(gateim.GetSize())/2.0))
print(gateim.GetSize())
tx = rdim.GetSize()[0]-gateim.GetSize()[0]
print(tx)
ty = rdim.GetSize()[1]-gateim.GetSize()[1]
tz = 0
euler3d.SetTranslation((tx, ty, tz))
print(gateim.GetWidth(),0, 0)
print
print('bonjour')

print('O ', gateim.TransformIndexToPhysicalPoint((0,0,0)))
print('A ', gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),0,0)))
print('B ', gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),gateim.GetHeight(),0)))
print('C ', gateim.TransformIndexToPhysicalPoint((0,gateim.GetHeight(),0)))
print('D ', gateim.TransformIndexToPhysicalPoint((0,0,gateim.GetSize()[2])))
print('E ', gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),0,gateim.GetSize()[2])))
print('F ', gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),gateim.GetHeight(),gateim.GetSize()[2])))
print('G ', gateim.TransformIndexToPhysicalPoint((0,gateim.GetHeight(),gateim.GetSize()[2])))
O=gateim.TransformIndexToPhysicalPoint((0,0,0))
A=gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),0,0))
B=gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),gateim.GetHeight(),0))
C=gateim.TransformIndexToPhysicalPoint((0,gateim.GetHeight(),0))
D=gateim.TransformIndexToPhysicalPoint((0,0,gateim.GetSize()[2]))
E=gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),0,gateim.GetSize()[2]))
F=gateim.TransformIndexToPhysicalPoint((gateim.GetWidth(),gateim.GetHeight(),gateim.GetSize()[2]))
G=gateim.TransformIndexToPhysicalPoint((0,gateim.GetHeight(),gateim.GetSize()[2]))
print(O)
print(A)
print(B)
print(C)
print(D)
print(E)
print(F)
print(G)
extreme_points = [O,A,B,C,D,E,F,G]
inv_euler3d = euler3d.GetInverse()
#print (extreme_points[0])                
#print(inv_euler3d.TransformPoint(extreme_points[0]))
extreme_points_transformed = [inv_euler3d.TransformPoint(pnt) for pnt in extreme_points]
min_x = min(extreme_points_transformed)[0]
min_y = min(extreme_points_transformed, key=lambda p: p[1])[1]

max_x = max(extreme_points_transformed)[0]
max_y = max(extreme_points_transformed, key=lambda p: p[1])[1]

# # Use the original spacing (arbitrary decision).
output_spacing = gateim.GetSpacing()
print(output_spacing)
# # Identity cosine matrix (arbitrary decision).
output_direction = [1.0, 0.0, 0.0, 1.0]
# # Minimal x,y coordinates are the new origin.
output_origin = [min_x, min_y]
print(output_direction)
# # Compute grid size based on the physical size and spacing.
output_size = [int((max_x-min_x)/output_spacing[0]), int((max_y-min_y)/output_spacing[1])]
print(output_size)
# resampled_image = sitk.Resample(logo, output_size, euler2d, sitk.sitkLinear, outp
# ut_origin, output_spacing, output_direction)
# plt.imshow(sitk.GetArrayViewFromImage(resampled_image))
# plt.axis('off')
# plt.show()


(134.57, 128.215, -180.0)
(184.3847655, 119.3847655, -180.0)
(50, 50, 81)
97
50 0 0
bonjour
O  (134.57, 128.215, -180.0)
A  (384.57, 128.215, -180.0)
B  (384.57, 378.21500000000003, -180.0)
C  (134.57, 378.21500000000003, -180.0)
D  (134.57, 128.215, 225.0)
E  (384.57, 128.215, 225.0)
F  (384.57, 378.21500000000003, 225.0)
G  (134.57, 378.21500000000003, 225.0)
(134.57, 128.215, -180.0)
(384.57, 128.215, -180.0)
(384.57, 378.21500000000003, -180.0)
(134.57, 378.21500000000003, -180.0)
(134.57, 128.215, 225.0)
(384.57, 128.215, 225.0)
(384.57, 378.21500000000003, 225.0)
(134.57, 378.21500000000003, 225.0)
(5.0, 5.0, 5.0)
[1.0, 0.0, 0.0, 1.0]
[50, 50]


In [18]:
rdpixres2=rdpixres
mask=np.ma.make_mask(rdpixres)
print(mask.shape)
print(gatepix.shape)

gatepix_mask=gatepix*mask
#print(gatepix_mask.shape)
ct_viewer(gatepix_mask)
ct_viewer(rdpixres)

(81, 50, 50)
(81, 50, 50)


interactive(children=(IntSlider(value=40, description=u'myslice', max=81), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=40, description=u'myslice', max=81), Output()), _dom_classes=('widget-interact',))

In [19]:
gate_result=sitk.GetImageFromArray(gatepix_mask)
print(gate_result.GetOrigin())
print(gate_result.GetSpacing())
print(gate_result.GetSize())
gate_result.SetOrigin(rdim_res.GetOrigin())
gate_result.SetSpacing(rdim_res.GetSpacing())
print(gate_result.GetOrigin())
print(gate_result.GetSpacing())
print(gate_result.GetSize())
sitk.WriteImage(gate_result, 'gate_result.mhd')

(0.0, 0.0, 0.0)
(1.0, 1.0, 1.0)
(50, 50, 81)
(-134.57, -128.215, -180.0)
(5.0, 5.0, 5.0)
(50, 50, 81)


In [20]:
print(rd)
#rp=dcm.read_file('RP.nose_9e.dcm')
#print(rp)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20180219'
(0008, 0013) Instance Creation Time              TM: '160638.372000'
(0008, 0016) SOP Class UID                       UI: RT Dose Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.7.296272965963.221760.20180219153417
(0008, 0020) Study Date                          DA: '20170828'
(0008, 0030) Study Time                          TM: '154812'
(0008, 0050) Accession Number                    SH: '103489158'
(0008, 0060) Modality                            CS: 'RTDOSE'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'GRAFF-CAILLEAUD^PIERRE'
(0008, 1010) Station Name                        SH: 'SRVRTDB'
(0008, 1030) Study Description                   LO: 'TDM therapie APC'
(0008, 103e) Series Description                  LO: 'Doses Eclipse'
(0008, 1048) P

In [22]:
print(rd.PixelSpacing)
rd.PixelSpacing=['5', '5']
print(rd.PixelSpacing)
#print(rd.Rows)
rd.Rows=50
rd.Columns=50
print(rd.Rows)
print(rd.Columns)
print (type(rd.GridFrameOffsetVector))
vect=np.arange(0, 405, 5)  #a faire jusqu'a -1
#print(vect)

vect[0]=str(vect[0])
print(type(vect[0]))


vect2=[]
for i in range(len(vect)):
    vect2.append(str(vect[i]));
    
#print(len(vect2))
rd.GridFrameOffsetVector=vect2
rd.NumberOfFrames=len(vect2)
print (rd.GridFrameOffsetVector)

print(gatepix_mask.shape)
ct_viewer(gatepix_mask)

print()
Matrix=np.zeros(((147, 90, 154)))


lowx=(Matrix.shape[0] // 2) - (gatepix_mask.shape[0] //2) -1
upx= (Matrix.shape[0] // 2) + (gatepix_mask.shape[0] //2)
lowy=(Matrix.shape[1] // 2) - (gatepix_mask.shape[1] //2)
upy= (Matrix.shape[1] // 2) + (gatepix_mask.shape[1] //2)
lowz=(Matrix.shape[2] // 2) - (gatepix_mask.shape[2] //2)
upz= (Matrix.shape[2] // 2) + (gatepix_mask.shape[2] //2)

Matrix[lowx:upx, lowy:upy, lowz:upz]=gatepix_mask
#Matrix=Matrix.astype(np.uint8)

ct_viewer(Matrix)

rd.PixelData = Matrix.tostring()
rd.save_as("gate.dcm")
rg=dcm.read_file('gate.dcm')
#print(rg)
gpix=rg.pixel_array.astype(np.float32)
#ct_viewer(rgpix)

['5', '5']
['5', '5']
50
50
<type 'list'>
<type 'numpy.int64'>
['0', '5', '10', '15', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95', '100', '105', '110', '115', '120', '125', '130', '135', '140', '145', '150', '155', '160', '165', '170', '175', '180', '185', '190', '195', '200', '205', '210', '215', '220', '225', '230', '235', '240', '245', '250', '255', '260', '265', '270', '275', '280', '285', '290', '295', '300', '305', '310', '315', '320', '325', '330', '335', '340', '345', '350', '355', '360', '365', '370', '375', '380', '385', '390', '395', '400']
(81, 50, 50)


interactive(children=(IntSlider(value=40, description=u'myslice', max=81), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=73, description=u'myslice', max=147), Output()), _dom_classes=('widget-interact',))

ValueError: cannot reshape array of size 4074840 into shape (81,50,50)

In [54]:
rd=dcm.read_file('rd_9e.dcm')
print(rd)
Matrix=np.zeros(((154, 90, 147)))


lowx=(Matrix.shape[0] // 2) - (gatepix_mask.shape[0] //2) -1
upx= (Matrix.shape[0] // 2) + (gatepix_mask.shape[0] //2)
lowy=(Matrix.shape[1] // 2) - (gatepix_mask.shape[1] //2)
upy= (Matrix.shape[1] // 2) + (gatepix_mask.shape[1] //2)
lowz=(Matrix.shape[2] // 2) - (gatepix_mask.shape[2] //2)
upz= (Matrix.shape[2] // 2) + (gatepix_mask.shape[2] //2)

Matrix[lowx:upx, lowy:upy, lowz:upz]=gatepix_mask

Matrix=Matrix/1e-10
Matrix=Matrix.astype(np.int32)
ct_viewer(Matrix)
print(Matrix.max())
rd.PixelData = Matrix.tostring()
#print(rd)
rd.save_as("gate.dcm")
rg=dcm.read_file('gate.dcm')
#print(rg)
gpix=rg.pixel_array
ct_viewer(gpix)


(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20180219'
(0008, 0013) Instance Creation Time              TM: '160638.372000'
(0008, 0016) SOP Class UID                       UI: RT Dose Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.7.296272965963.221760.20180219153417
(0008, 0020) Study Date                          DA: '20170828'
(0008, 0030) Study Time                          TM: '154812'
(0008, 0050) Accession Number                    SH: '103489158'
(0008, 0060) Modality                            CS: 'RTDOSE'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'GRAFF-CAILLEAUD^PIERRE'
(0008, 1010) Station Name                        SH: 'SRVRTDB'
(0008, 1030) Study Description                   LO: 'TDM therapie APC'
(0008, 103e) Series Description                  LO: 'Doses Eclipse'
(0008, 1048) P

interactive(children=(IntSlider(value=77, description=u'myslice', max=154), Output()), _dom_classes=('widget-interact',))

1612746


interactive(children=(IntSlider(value=77, description=u'myslice', max=154), Output()), _dom_classes=('widget-interact',))

2147483648